<a href="https://colab.research.google.com/github/hantedyou/kccnlpClass/blob/main/NER_KCC_mod_auto_VM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import gc
import math
#import KCC_NER_Util

In [ ]:
from ckiptagger import construct_dictionary
import datetime
from ckiptagger import WS, POS, NER
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

In [ ]:
"""
執行完後應把相關資源釋放
"""
del WS
del POS
del NER
del ws
del pos
del ner
del dictionary
gc.collect()

634624

# 1.word_proc_dataframe : 將輸入的資料作斷詞、詞性標註及NER

## 1-1 載入自訂辭典，使用CKIPTagger的功能"construct_dictionary"

In [ ]:
# 載入 自訂詞典
User_Dict = {}
with open("CIS_Dict2.txt","r", encoding='utf-8-sig') as UDicts:
    for udic in UDicts:
        udWord = udic.strip().split(" ")
        if len(udWord) > 1:
            User_Dict[udWord[0]] = udWord[1]
        else:
            User_Dict[udWord[0]] = 10
dictionary = construct_dictionary(User_Dict)

## 1-2 定義資料處理module

In [ ]:
def word_proc_dataframe(input_Series,
              column_name = 'Text',
              tt_seg = (1,10),
              export_flag = False, 
              #tt_begin = 0,tt_end = 50,
              output_file = "df_test_out.csv"):
    """
        data_path(str):資料路徑(字串)
        column_name(str):要處理的欄位名稱
        tt_seg(tuple):起訖列號
        export_flag(boolean):是否產出檔案
        output_file:輸出的檔案名稱，指定為CSV檔    
    """
    
    
    """
    file_type = data_path[-4:]
    print("輸入檔案格式為 {}".format(file_type))
    if 'csv' in file_type:
        data = pd.read_csv(data_path)
    elif 'xls' in file_type:
        data = pd.read_excel(data_path)
    else:
        #print("檔案須為CSV或XLS/XLSX")
        raise Exception("檔案須為CSV或XLS/XLSX")
        #return None
    """
    
    """
    column_list = list(input_df.columns)
    if column_name in column_list:
        text = input_df[column_name]
    else:
        print("輸入欄位\"{input_column}\"不存在\n匯入檔案欄位{i_column_name}".format(input_column = column_name,
                                                                       i_column_name=str(column_list)))
        raise Exception("不存在此欄位名稱")
    """
    
    #ls_Test = list(text[tt_begin:tt_end])
    
    ###########      debug        ###############
    
    print("起始列數：{begin}\n結束列數:{end}".format(begin = (tt_seg[0]),end = tt_seg[1]))
    print("列印前3筆:\n{}".format(input_Series[:3]))
    print("列印末3筆:\n{}".format(input_Series[-3:]))
    
    #print(input_Series[:10])
    #print(input_Series[tt_seg[0]:tt_seg[1]])
    ###########      debug        ###############
    
    ls_len = len(input_Series)
    ls_Test = list(input_Series[:])
    
    
    
    if (len(ls_Test) > 0):
        print("取得list，長度為：{list_len}\n 載入 WS 資料...".format(list_len = len(ls_Test)))
    else:
        raise Exception("資料長度為0")
        
    
    print("============================\n 開始執行 WS\n============================\n")
    # 紀錄 WS 起始時間
    startTime_WS = datetime.datetime.now()   
    print(startTime_WS)
    
    word_sentence_list = ws(ls_Test,
                                # sentence_segmentation=True, # To consider delimiters
                                # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}),
                                # This is the defualt set of delimiters
                                # recommend_dictionary = dictionary, # words in this dictionary are encouraged
                                coerce_dictionary = dictionary, # words in this dictionary are forced
                            )
    
    # 紀錄 WS 結束時間
    endTime_WS = datetime.datetime.now()
    # 計算本次 WS 執行時間
    exeTime_WS = endTime_WS - startTime_WS    
    print("============================\n WS 完成\n============================\n")
    print("\t\t WS 執行時間:  {}".format(exeTime_WS))
    
    
    print("============================\n 開始執行 POS \n============================\n")
    # 紀錄 POS 起始時間
    startTime_POS = datetime.datetime.now()   
    print(startTime_POS)
    

    pos_sentence_list = pos(word_sentence_list)
    
    #紀錄 POS 結束時間
    endTime_POS = datetime.datetime.now()
    #計算本次 POS 執行時間
    exeTime_POS = endTime_POS - startTime_POS    
    
    print("============================\n POS 完成\n============================\n")
    print ("\t\t POS 執行時間:  {}".format(exeTime_POS))
    
    
    print("============================\n 開始執行 NER \n============================\n")
    #紀錄 NER 起始時間
    startTime_NER = datetime.datetime.now()   
    print(startTime_NER)
    
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    
    #紀錄 NER 起始時間
    endTime_NER = datetime.datetime.now()
    #計算本次 NER 執行時間
    exeTime_NER = endTime_NER - startTime_NER    
    print("============================\n NER 完成\n============================\n")
    print ("\t\t NER 執行時間:  {}".format(exeTime_NER))
    
    """
    建立一個DataFrame物件，
    其結構為['原始資料','WS結果','POS結果','NER結果']
    """
    
    df_Test = pd.DataFrame(ls_Test,columns=[column_name])
    df_Test['WS'] = np.array(word_sentence_list)
    
    ##################
    #print("DEBUG")
    #print(pos_sentence_list[:5])
    ##################
    
    df_Test['POS'] = np.array(pos_sentence_list)
    df_Test['NER'] = np.array(entity_sentence_list)
    print("============================\n Dataframe Created \n============================\n")
    
    if export_flag is True:
        df_Test.to_csv(output_file)
        print("File Exported!!!!!")
    else:
        print("斷詞處理完成，不產生檔案")
        
    #return df_Test,word_sentence_list,pos_sentence_list,entity_sentence_list
    
    #回傳 NER 結果
    return entity_sentence_list


# 2.gen_df:產生df["類別","NER"]供產檔使用

In [ ]:
def gen_df(in_list):
    """
    1.功能：將中研院 ckiptagger.NER 處理後之NER資料(list)打開，
    取出NER類別及專有名詞，分別存到df_class_list 及df_ner_list中，
    並將其組成有兩個欄位['類別','專有名詞']的DataFrame。
    
    2.說明：
    中研院 ckiptagger.NER 處理後之NER資料有N個set，
    N為輸入資料的筆數，
    每個set表示該段文字取出之NER。
    因為set沒有順序性，不好處理，
    把set群轉成ner_list有N個set；
    ner_list有M個tuple，
    每個代表一個字的NER
    EX:(475, 482, 'ORG', '臺中國家歌劇院')。
    """
    df_class_list = list()
    df_ner_list = list()
    
    count = 0
    for i in range(len(in_list)):
        for j in range(len(in_list[i])):
            count = count +1
            a = in_list[i].pop()
            df_class_list.append(a[2])    #NER類別
            df_ner_list.append(a[3])      #NER
        #print(type(new_list[j]),(new_list[j]))
    #print(count)
    extract_df = pd.DataFrame({'類別':df_class_list,'專有名詞':df_ner_list})
    
    #刪除重複
    dd_extract_df = extract_df.drop_duplicates(subset ="專有名詞").copy()
    del extract_df
    #df_exp = dd_extract_df.loc[dd_extract_df['類別'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE'])].copy()
    
    #取出需要的類別
    df_exp = dd_extract_df.loc[dd_extract_df['類別'].isin(['EVENT','FAC','GPE','LAW','LOC','MONEY','NORP','ORG','PERSON'])].copy()
    del dd_extract_df
    
    #刪除長度不到2的資料
    df_exp_2 = df_exp[df_exp['專有名詞'].map(len) >= 2].copy()
    del df_exp
    
    return df_exp_2
    

# 3.proc_seg : 自動處理NER及產生CSV檔

In [ ]:
def proc_seg(in_df,export_path,seg_len=100):
    """
    將資料分段處理，
    in_df:要分段完整資料
    seg_len:每一段要處理的長度(幾row)
    export_path:檔案匯出路徑
    """
    
    #計算輸入資料長度
    df_length = len(in_df)
    #計算要分成幾段處理
    seg_n = math.ceil(df_length/seg_len) 
    
    print("輸入資料長度:{in_len}\n共分成{seg}段".format(in_len = df_length,seg = seg_n))
    
    """
    產生一個字典物件obj，
    其內容為分段後之資料，
    每一個key 'df_i'對應到一個長度為seg_len的Series，
    Series內每一列即為一篇文字(string)。
    """
    obj = {}
    for i in range(seg_n):
        #print(i*seg_len,(i+1)*seg_len)
        #print(df_seg_test[(i*100):(i+1)*100])
        obj['df_'+str(i)] = in_df[(i*seg_len):(i+1)*seg_len]   
        
    """
    1.將分段資料丟到 word_proc_dataframe 及 gen_df 處理。
    2.word_proc_dataframe : 產生NER結果(list)並回傳。
    3.gen_df : 將 word_proc_dataframe 產出的list內容取出，轉換成DataFrame['類別','專有名詞(NER)']
    tt_seg邏輯：
        segment = 0
        tt_seg = (0,100)

        segment = 1
        tt_seg = (100,200)
        ...
        segment = n
        tt_seg = (n,n*seg_len)
    """
    for segment in range(len(obj)):
        
        print("=======================\t開始處理第{}段\t===================\n".format(segment))
        print("第{}段長度：{}".format(('df_'+ str(segment)),len(obj['df_'+ str(segment)])))
        
        ########################################################################################
        # 呼叫word_proc_dataframe，回傳NER list
        ########################################################################################
        
        print("(segment*seg_len:{},(segment+1)*segment):{}".format((segment*seg_len),(segment+1)*seg_len))
        print("=====================================\b呼叫word_proc_dataframe\b=============================\n")
        entity_list = word_proc_dataframe(input_Series = obj['df_'+ str(segment)],
                            tt_seg = ((segment*seg_len),(segment+1)*seg_len))
        
        print("=====================================呼叫 gen_df=============================\n")
        temp_df = gen_df(entity_list)
        #temp_df.sort_values(by=['類別']).to_csv("NER_Export_New\df_out_" + str(segment) + ".csv")
        temp_df.sort_values(by=['類別']).to_csv(export_path + str(segment) + ".csv")
        #print(type(obj['df_'+ str(segment)]))
        print("===========================  第{}段處理完成 ===========================  \n".format(segment))
        
        #print(obj['df_'+ str(segment)])
    gc.collect()

# 4.讀取檔案及執行自動產檔

In [ ]:
#讀檔 新聞
full_path = r"剪報系統匯出-正確\all-new(11554).xlsx"
df_full_text = pd.read_excel(full_path)
df_full_text.columns

Index(['序號', '日期', '報別', '內容'], dtype='object')

In [ ]:
#讀檔 議事資料
# df_file = pd.read_csv(r"News Data\News11554.csv")
df_full_text = pd.read_csv(r"Johnson\00 KCC NER\CIS Data\CisPdf2CsvByFile.csv")
print(df_full_text.columns)
print("-----------------------------------------------------------")
df_full_text.head(3)

Index(['FileNo', 'FileName', 'Text'], dtype='object')
-----------------------------------------------------------


,FileNo,FileName,Text
0,1,1-1-1000516-1-蘇炎城.pdf,一、第 1 屆第 1 次定期大會第 27 次會議 （中華民國 100 年 5 月 16 日上...
1,2,1-1-1000516-2-張勝富.pdf,主席（許議長崑源）： 繼續開會，請張議員勝富質詢。 張議員勝富： 大會主席、陳市長、市政府一...
2,3,1-1-1000516-3-吳益政.pdf,議長崑源）： 繼續開會，請吳議員益政質詢。 吳議員益政： 主席、市長、三位副市長還有各位局...


In [ ]:
#df_seg_test = df_full_text["Text"]
df_seg_test = df_full_text["內容"][:100]
ex_path = r"NER_Export_New\Test\\"
# 執行產檔
starttime = datetime.datetime.now()    #紀錄執行時間
print("開始執行NER作業，起始時間{}".format(starttime))
proc_seg(df_seg_test,export_path=ex_path, seg_len=30)
endtime = datetime.datetime.now()    #紀錄執行時間
exeTime = endtime - starttime
print ("NER作業執行完成，執行時間：{}".format(exeTime))

開始執行NER作業，起始時間2020-02-11 10:33:57.664125
輸入資料長度:100
共分成4段
=======================	開始處理第0段	===================

第df_0段長度：30
(segment*seg_len:0,(segment+1)*segment):30
====================================呼叫word_proc_datafram=============================

起始列數：0
結束列數:30
列印前3筆:
0    馬英九總統昨天上午視察衛武營藝術文化中心，文化部長洪孟啟、工程會副主委顏久榮及高雄市副市長吳...
1    　日本九州熊本縣接連遭強震襲擊，台灣分別捐贈熊本縣政府和日本政府款項，高雄市長陳菊、台南市 ...
2    　壽山動物園為增進動物飼養福祉及保育工作，特設立「高雄市壽山動物園認養辦法」設立動物認養專戶...
Name: 內容, dtype: object
列印末3筆:
27    　針對高雄市議會國民黨團昨天指責高雄市政府徵收污水下水道使用費還要民眾自行檢驗，向市府提出鑑...
28    　市議員林瑩蓉昨天針對經濟發展局業務報告提出楠梓創新研發園區構想，感到憂心，呼籲楠梓中油廠區...
29    　行政院評估高雄市大林蒲遷村至少要支出七百億元，高雄市長陳菊日前表示，該案是經濟部的評估 ，...
Name: 內容, dtype: object
取得list，長度為：30
 載入 WS 資料...
 開始執行 WS

2020-02-11 10:33:57.669123
 WS 完成

		 WS 執行時間:  0:00:16.976074
 開始執行 POS 

2020-02-11 10:34:14.645197
 POS 完成

		 POS 執行時間:  0:00:13.624369
 開始執行 NER 

2020-02-11 10:34:28.270566
 NER 完成

		 NER 執行時間:  0:00:19.066001
 Dataframe Created 

斷詞處理完成，不產生檔案
=====================================呼叫 gen_df===

In [ ]:
#選擇要的欄位
#df_seg_test = df_full_text["內容"] #新聞
#df_seg_test = df_full_text["Text"]
# 執行產檔
#proc_seg(df_seg_test,50)

# 5 將所有檔案彙整成一個CSV，刪掉重複&案類別排序後匯出

## 5-1 彙整檔案

In [ ]:
import glob
import pandas as pd

path = ex_path 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

C:\Users\kccadmin\Anaconda3\envs\Data_science\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [ ]:
print(len(frame))
frame.groupby('類別').count()

1128


,專有名詞
類別,
EVENT,24
FAC,71
GPE,154
LAW,20
LOC,42
MONEY,81
NORP,20
ORG,403
PERSON,313


## 5-2 取出需要的類別

In [ ]:
df_exp_all = frame.loc[frame['類別'].isin(['EVENT','FAC','GPE','LAW','LOC','NORP','ORG','PERSON'])].copy()
print(len(df_exp_all))
df_exp_all.groupby('類別').count().to_csv(path + "\statistics.csv")
print(len(df_exp_all))

1047
1047


## 5-3 去除字串空白

In [ ]:
#去空白
print(len(df_exp_all))
df_exp_all['專有名詞'] = df_exp_all['專有名詞'].str.strip()    #去前後空白
print(len(df_exp_all))
df_exp_all['專有名詞'] = df_exp_all['專有名詞'].str.replace(' ', '')   #去中間空白
print(len(df_exp_all))
df_exp_all.groupby('類別').count().to_csv(path + "\statistics.csv")

1047
1047
1047


## 5-4 刪除重複：
###    因為空白可能在字串不同地方，所以要先去空白再刪重複 

In [ ]:
df_exp_all = df_exp_all.drop_duplicates().copy()
print(len(df_exp_all))
df_exp_all.groupby('類別').count().to_csv(path + "\statistics.csv")

853


## 5-5 將處理完的DataFrame匯出檔案

In [ ]:
# 匯出檔案
df_exp_all.sort_values(by=['類別']).to_csv(r"NER_Export_New\df_out_all.csv")

# ========================================    
# 結束   
# ========================================

# 2020 02 11後續處理
## 全形轉半形
## 將類別改為中文

In [ ]:
#path = "NER_Result\\"
path = r"NER_Result\NER_Combine_Exp\\"
df_continue = pd.read_excel(path + "20200211-ner.xlsx")
#df_continue = pd.read_csv(path + "sortbyNERLenCat_NBOM_test.csv")


In [ ]:
import datetime


In [ ]:
a = str(datetime.datetime.now()).replace("-","_")
print(str(a)[:10].replace("-","_"))

2020_02_11


In [ ]:
print(df_continue.columns)
#df_continue = df_continue[['類別','專有名詞','length']]
#print(df_continue.columns)

Index(['編號', '類別', '專有名詞', 'length'], dtype='object')


In [ ]:
# 全形轉半形
df_continue['專有名詞'] = df_continue['專有名詞'].str.normalize('NFKC')

In [ ]:
# 刪除特殊符號
# https://stackoverflow.com/questions/50444346/fast-punctuation-removal-with-pandas
df_continue['專有名詞'] = df_continue['專有名詞'].str.replace(r'[^\w\s]+', '')

In [ ]:
df_continue['類別'] = df_continue['類別'].replace('ORG', '組織').replace('PERSON', '人物').replace('GPE', '行政區域').replace('LOC', '地理區域').replace('EVENT', '事件').replace('LAW', '法律').replace('FAC', '設施')

In [ ]:
df_continue.to_excel(path + "NER_"+ str(datetime.datetime.now())[:10].replace("-","_")+ ".xls",sheet_name= "NER_result")

In [ ]:
df_continue.to_csv(path + "sortbyNERLenCat_NBOM_20200211.csv")

In [ ]:
#https://www.geeksforgeeks.org/python-pandas-series-str-translate/
trans_dict ={"PERSON":"人物","GPE":"行政區域","LOC":"地理區域","EVENT":"事件","LAW":"法律","NORP":"民族宗教政治團體"}

In [ ]:
path = "NER_Result\\"
path = r"NER_Export_New\Test\\"

In [ ]:
a = pd.read_csv(path + "3.csv")

In [ ]:
print(a.columns)
a.head(10)

Index(['Unnamed: 0', '類別', '專有名詞'], dtype='object')


,Unnamed: 0,類別,專有名詞
0,23,EVENT,美麗島１２３
1,25,GPE,ａｄｆｇ
2,73,GPE,美麗
3,74,GPE,Ｖｅｉｓｊｒｈｌｋｊｌｋｊｉｏｊｌｋｊ
4,75,GPE,Ｒｕｓｓｉａ
5,53,GPE,美麗島
6,35,GPE,大順路
7,147,GPE,高雄市
8,71,GPE,巴拿馬
9,24,GPE,林肯


In [ ]:
s = 'one two one two one'

In [ ]:
print(s.replace('one', 'XXX'))
# XXX two XXX two XXX

print(s.replace('one', 'XXX', 2))
# XXX two XXX two one

XXX two XXX two XXX
XXX two XXX two one


In [ ]:
print(s.replace('one', 'XXX').replace('two', 'YYY'))

XXX YYY XXX YYY XXX
